## PHAS0113 Group E: Exposure time calculations
Author: Zuyi Wakami Soh 22/01/2023

In [15]:
import numpy as np

In [37]:
# The code for all the calculations have been adapted from ExposureTimes.py (Powell, 2022)
mag, zdegrees= np.loadtxt("Target_Data.txt", unpack= True)
aperture, T, R, eta, bandwidth, msky, Ssky, Ndark, npix= np.loadtxt("Telescope_Data.txt", unpack= True)

z = zdegrees/360*np.pi*2 # zenith distance in radians

In [38]:
# Equations for calculations obtained from PHAS0043 Expt 1: Observing Programme (Boyle & Fossey, 2022)
def atmo_extinction(k, z):
    """
    Calculates the atmospheric extinction, a   
    - a=10^{-0.4*k*X}
    
    Arguments:
    - k: wavelength-dependent extinction coefficient
    - z: zenith distance in radians
    
    Returns:
    - Atmospheric extinction, a
    """
    X = 1/np.cos(z) 
    return 10 ** (-0.4 * k * X) 

def sourceflux(mag):
    """
    Calculates the source flux, F_S (photons/sec/cm^2/Angstrom) at the top of the Earth's atmosphere
    - F_S=F_{Vega} * 10^{-0.4*(m_S-m_Vega)}
    
    Arguments:
    - mag: magnitude of source
    
    Returns:
    - Source flux
    """
    Fvega = 1000
    mvega = 0
    return Fvega * 10 ** (-0.4 * (mag - mvega))

def source_countrate(mag, k, z, aperture, T, R, eta, bandwidth):
    """
    Calculate detected photo-electron flux from the source, \dot{N}_S.
    - \dot{N}_S =F_S * a * A * T * R * \eta * \delta \lambda (electrons/s)
    
    Arguments:
    - mag: magnitude of source
    - k: wavelength-dependent extinction coefficient
    - z: zenith distance in radians
    - aperture: aperture of telescope in cm
    - T: transmission efficiency of telescope optics
    - R: transmission efficiency of filter
    - eta: CCD quantum efficiency
    - bandwidth: effective bandwith of filter, \delta \lambda
    
    Returns:
    - \dot{N}_S: detected photo-electron flux from the source
    """
    F = sourceflux(mag)
    a = atmo_extinction(k, z)
    A = np.pi * (aperture/2) ** 2
    return F * a * A * T * R * eta * bandwidth

def sky_countrate(msky, Ssky, aperture, T, R, eta, bandwidth):
    """
    Calculate the photo-electron flux due to the sky, \dot{N}_{sky}
    - \dot{N}_{sky} =F_{sky} * A * T * R * eta * \delta \lambda * s_{sky}
    
    Arguments:
    - msky: brightness of the night sky, m_{sky} (magnitudes per square arcsec)
    - Ssky: number of square arcseconds over which the source is recorded on the detector, s_{sky}
    - aperture: aperture of telescope in cm
    - T: transmission efficiency of telescope optics
    - R: transmission efficiency of filter
    - eta: CCD quantum efficiency
    - bandwidth: effective bandwith of filter, \delta \lambda
    
    Returns:
    - \dot{N}_{sky}: photo-electron flux due to the sky
    """
    F = sourceflux(msky)
    A = np.pi * (aperture/2) ** 2
    return F * A * T * R * eta * bandwidth * Ssky

def background_countrate(sky_countrate, dark_countrate, npix):
    """
    Calculates the photo-electron flux due to the background, \dot{N}_B
    
    Arguments:
    - sky_countrate: photo-electron flux due to the sky
    - dark_countrate: dark current per pixel
    - npix: number of pixels over which the source signal is integrated
    
    Returns:
    - \dot{N}_B: photo-electron flux due to the background
    """
    return sky_countrate + dark_countrate * npix
    

def expotime(SNR, source_countrate, sky_countrate):
    """
    Calculates the exposure time for a given SNR.
    
    Arguments:
    - SNR: signal-to-noise ratio
    - source_countrate: detected photo-electron flux from the source, \dot{N}_S
    - sky_countrate: photo-electron flux due to the sky
    
    Returns:
    - t: exposure time
    """
    return (SNR ** 2)*(source_countrate + 2 * sky_countrate)/(source_countrate**2)

In [39]:
k= 0.2
Ns = source_countrate(mag, k, z, aperture, T, R, eta, bandwidth)
Nb = background_countrate(sky_countrate(msky, Ssky, aperture, T, R, eta, bandwidth), Ndark, npix)

SNR30 = Ns*30/np.sqrt(Ns*30+2*Nb*30)
SNR60 = Ns*60/np.sqrt(Ns*60+2*Nb*60)

print("SNR for exposure time of 30s:",SNR30)
print("SNR for exposure time of 60s:",SNR60)

SNR for exposure time of 30s: [ 2538.2201536   1561.57835831  7757.33721221 12706.82159532
  4709.40023837  9208.6449418   9592.51204104  8469.85277531
 18329.14878083  6065.54821841]
SNR for exposure time of 60s: [ 3589.58536551  2208.40529302 10970.53149341 17970.15943475
  6660.09768774 13022.99056777 13565.86062567 11978.18066614
 25921.3307926   8577.9805537 ]


In [40]:
tHZHer= expotime(1000, Ns[0], Nb[0])
print("Minimum exposure time for HZ Her: ", tHZHer, "s")
tXYCVn= expotime(1000, Ns[1], Nb[1])
print("Minimum exposure time for XY CVn: ", tXYCVn, "s")
tRRGem= expotime(1000, Ns[2], Nb[2])
print("Minimum exposure time for RR Gem: ", tRRGem, "s")
tRRBoo= expotime(1000, Ns[3], Nb[3])
print("Minimum exposure time for RR Boo: ", tRRBoo, "s")
tNNBoo= expotime(1000, Ns[4], Nb[4])
print("Minimum exposure time for NN Boo: ", tNNBoo, "s")
tHV7558= expotime(1000, Ns[5], Nb[5])
print("Minimum exposure time for HV 7558: ", tHV7558, "s")
tTVLyn= expotime(1000, Ns[6], Nb[6])
print("Minimum exposure time for TV Lyn: ", tTVLyn, "s")
tHV7457= expotime(1000, Ns[7], Nb[7])
print("Minimum exposure time for HV 7457: ", tHV7457, "s")
tHV7560= expotime(1000, Ns[8], Nb[8])
print("Minimum exposure time for HV 7560: ", tHV7560, "s")
tHV7596= expotime(1000, Ns[9], Nb[9])
print("Minimum exposure time for HV 7596: ", tHV7596, "s")

Minimum exposure time for HZ Her:  4.656532929606273 s
Minimum exposure time for XY CVn:  12.302509006380951 s
Minimum exposure time for RR Gem:  0.498535298109409 s
Minimum exposure time for RR Boo:  0.18580071881798849 s
Minimum exposure time for NN Boo:  1.3526643738156314 s
Minimum exposure time for HV 7558:  0.353777165797751 s
Minimum exposure time for TV Lyn:  0.32602923780266474 s
Minimum exposure time for HV 7457:  0.41818604089444444 s
Minimum exposure time for HV 7560:  0.08929695745575068 s
Minimum exposure time for HV 7596:  0.815419580132987 s


### References

Powell, S. (2022) Samjpowell13/ExpoPy: Python code written for Task 1 of phas0043 at the UCL Observatory (group 2), GitHub. Available at: https://github.com/SamjPowell13/ExpoPy (Accessed: January 24, 2023).

Boyle, S. and Fossey, S. (2022) Expt 1: Observing Programmes (Group Exercise) for the UCLO Celestron and Telescope Live Robotic Telescopes. issue brief. London, London: UCL, pp. 12–14. 